In [1]:
import numpy as np
import random as rn
import findspark
import sys
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
#findspark.init("D:\\Users\\software\\spark-3.2.1-bin-hadoop3.2\\bin\\..")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 10
W= 1.0
c_1 = 0.3
c_2 = 0.7
objetivo =[50,50,50]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 1
posiciones_=[]
mejor_posiciones_locales=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales])
mejor_pos_global= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max

In [6]:
def InitParticles(part,N,M,bgf,blf):
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(n)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales=posiciones.copy()
        velocidad=[rn.uniform(-100,100) for j in range(n)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("best local fitness",blf)
        if fit < bgf:
            bgf = fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,bgf,mejor_posiciones_locales,mejor_pos_global,part 

In [7]:
def fitnessEval(part,mpl,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        mpl[ind] = part[0:1,ind]
        if fit < bgf:
            bgf = fit
            mpg = part[0:1,ind]
    return part,mpl,mpg,blf,bgf

In [8]:
def posEval(part,mpl,mpg,ind):
    velocidades=part[1:2,ind][0]
    for k in range(n):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[ind][k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
    return part,velocidades

In [9]:
best_local_fitness,best_global_fitness,mejor_posiciones_locales,mejor_pos_global,particulas = InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
for i in range (I):
    for ind in range(m):
        particulas,mejor_posiciones_locales,mejor_pos_global,best_local_fitness,best_global_fitness=fitnessEval(particulas,mejor_posiciones_locales,mejor_pos_global,best_local_fitness,best_global_fitness,ind)
    r_1 = rn.random()
    r_2 = rn.random()
    for ind in range(m):
        particulas,velocidades = posEval(particulas,mejor_posiciones_locales,mejor_pos_global,ind)
print("Posiciones finales->",particulas[0:1][0])

Posiciones finales-> [[  3.75876994 -22.1737115  -16.53171886]
 [ 85.05581339 -31.8132174   72.01192053]
 [-54.11534231 -89.19090197 -71.9203159 ]
 [ 39.71186088  77.69261312 -26.77781235]
 [ 31.19868685 -88.72457641 -60.06589079]
 [-79.90288622 -21.00236652  86.0048714 ]
 [-81.27423235  -8.55119459  43.05757922]
 [ 86.25124533  -6.71328618 -44.54690275]
 [-56.39647314 -59.3229727    7.73065186]
 [-62.89778354  63.31024466  89.0408576 ]]


In [10]:
#rdd_master= sc.parallelize(particulas)